In [82]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model
import tensorflow as tf

from sklearn.impute import KNNImputer


In [83]:
import pandas as pd


df = pd.read_csv('Final_ext_compositions_w_subcategories.csv')
df

,Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO2,MnO,K2O,...,LOI,Caption,DOI,Descriptor,Response,Category,Is_Outlier,total,Year,Sub_category
0,Zeolitic rock from Sakhaptinskoje deposit,64.80,12.77,2.46,NaN,1.84,NaN,0.35,NaN,NaN,...,11.12,Chemical composition of primary material.,10.1016/j.conbuildmat.2009.10.010,The Zeolitic rock from Sakhaptinskoe deposit ...,"$Zeolite, Sakhaptinskoje deposit, Mineral$",Natural pozzolans,False,79.41,2009.0,zeolite
1,Ash microspheres,62.30,23.40,4.00,NaN,4.21,NaN,1.3,NaN,NaN,...,1.74,Chemical composition of primary material.,10.1016/j.conbuildmat.2009.10.010,Ash microspheres are spherical particles prim...,"$Ash, microspheres, Spherical particles, Silic...",Other ashes,False,89.91,2009.0,Other
2,Vermiculite,34.00,12.00,6.00,NaN,13.00,NaN,NaN,NaN,4.000,...,NaN,Chemical composition of primary material.,10.1016/j.conbuildmat.2009.10.010,Vermiculite is a mineral composed primarily o...,"$Vermiculite, Mineral$",Natural pozzolans,False,59.00,2009.0,vermiculite
3,HAC,0.20,71.00,0.10,28.50,0.40,NaN,NaN,NaN,NaN,...,0.5,Oxide composition and chemical properties of H...,10.1016/j.conbuildmat.2010.01.022,The material named 'HAC' contains approximate...,"$Cement, Calcium aluminate$",Cements,True,100.10,2010.0,Calcium aluminate
4,OPC,22.10,5.00,3.00,63.80,1.60,2,NaN,NaN,0.640,...,1.2,Oxide composition and chemical properties of H...,10.1016/j.conbuildmat.2010.01.022,The material named 'OPC' has an oxide composi...,"$Cement, OPC$",Cements,False,92.50,2010.0,CEM I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14424,natural FAp clinker,21.46,6.13,4.60,67.81,NaN,NaN,NaN,NaN,NaN,...,NaN,Expected elemental composition of the clinkers...,10.1016/j.cemconres.2018.01.006,Natural FAp clinker refers to clinker materia...,"$Clinker, Fluorapatite, Natural$",Clinkers,False,95.40,2018.0,Other
14425,J (CEM V/B (S-V)),25.00,NaN,NaN,25.00,50.00,NaN,NaN,NaN,NaN,...,NaN,Binders and its constituents.,10.1016/j.conbuildmat.2018.04.206,J (CEM V/B (S-V)) refers to a binder composed...,"$Cement, CEM V/B, S-V, 25% by mass$",Cements,False,100.00,2018.0,CEM V
14426,0-4 La Pallice sand,88.20,NaN,NaN,11.80,NaN,NaN,NaN,NaN,NaN,...,NaN,Properties of cements and limestone filler (LF).,10.1016/j.jclepro.2018.02.155,The material named '0-4 La Pallice sand' is a...,"$Aggregate, Sand, Siliceous, originating from ...",Other minerals,False,100.00,2018.0,Other
14427,0-4 Les Monards sand,92.50,NaN,NaN,7.50,NaN,NaN,NaN,NaN,NaN,...,NaN,Properties of cements and limestone filler (LF).,10.1016/j.jclepro.2018.02.155,The material named '0-4 Les Monards sand' is ...,"$Aggregate, Sand, Siliceous$",Other minerals,False,100.00,2018.0,Other


In [84]:
desired_columns=['SiO2', 'Al2O3', 'Fe2O3', 'CaO', 'MgO', 'SO3', 'TiO2', 'MnO', 'K2O',
       'Na2O', 'LOI', 'S gravity', 'Amorphous content']

In [85]:
# Create a new DataFrame with the desired columns
new_df = pd.DataFrame()

for column in desired_columns:
    if column in df.columns:
        # Add the existing column to the new DataFrame
        new_df[column] = df[column]
    else:
        # Case insensitive check and add if exists
        matches = [col for col in df.columns if col.lower() == column.lower()]
        if matches:
            new_df[column] = df[matches[0]]
        else:
            # Create a column with NaN values if it doesn't exist
            new_df[column] = np.nan

# Display the head of the new DataFrame to verify
print(new_df.head())

   SiO2  Al2O3  Fe2O3   CaO    MgO  SO3  TiO2  MnO   K2O  Na2O    LOI  \
0  64.8  12.77   2.46   NaN   1.84  NaN  0.35  NaN   NaN   NaN  11.12   
1  62.3  23.40   4.00   NaN   4.21  NaN   1.3  NaN   NaN   NaN   1.74   
2  34.0  12.00   6.00   NaN  13.00  NaN   NaN  NaN  4.00   NaN    NaN   
3   0.2  71.00   0.10  28.5   0.40  NaN   NaN  NaN   NaN   0.2    0.5   
4  22.1   5.00   3.00  63.8   1.60    2   NaN  NaN  0.64  0.35    1.2   

   S gravity  Amorphous content  
0        NaN                NaN  
1        NaN                NaN  
2        NaN                NaN  
3        NaN                NaN  
4        NaN                NaN  


In [86]:
new_df['d50']=10
new_df['Ca(OH)2']=3
new_df['CaCO3']=0
new_df['Water']=3.6
new_df['K2SO4']=0
new_df['KOH']=0.151
new_df['curing temp']=50
new_df['age']=10
new_df

,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO2,MnO,K2O,Na2O,...,S gravity,Amorphous content,d50,Ca(OH)2,CaCO3,Water,K2SO4,KOH,curing temp,age
0,64.80,12.77,2.46,NaN,1.84,NaN,0.35,NaN,NaN,NaN,...,NaN,NaN,10,3,0,3.6,0,0.151,50,10
1,62.30,23.40,4.00,NaN,4.21,NaN,1.3,NaN,NaN,NaN,...,NaN,NaN,10,3,0,3.6,0,0.151,50,10
2,34.00,12.00,6.00,NaN,13.00,NaN,NaN,NaN,4.000,NaN,...,NaN,NaN,10,3,0,3.6,0,0.151,50,10
3,0.20,71.00,0.10,28.50,0.40,NaN,NaN,NaN,NaN,0.2,...,NaN,NaN,10,3,0,3.6,0,0.151,50,10
4,22.10,5.00,3.00,63.80,1.60,2,NaN,NaN,0.640,0.35,...,NaN,NaN,10,3,0,3.6,0,0.151,50,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14424,21.46,6.13,4.60,67.81,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10,3,0,3.6,0,0.151,50,10
14425,25.00,NaN,NaN,25.00,50.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10,3,0,3.6,0,0.151,50,10
14426,88.20,NaN,NaN,11.80,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10,3,0,3.6,0,0.151,50,10
14427,92.50,NaN,NaN,7.50,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10,3,0,3.6,0,0.151,50,10


In [87]:
new_df['d50'][df['Category']=='Cements']=14.11
new_df['d50'][df['Category']=='Clinkers']=20
new_df['d50'][df['Category']=='Fly ashes']=8.88
new_df['d50'][df['Category']=='Slags']=6.15
new_df['d50'][df['Category']=='Glasses']=24.99


/tmp/ipykernel_2589503/534658166.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [88]:
new_df['d50'][df['Category']=='Silica fume']=0.2
new_df['S gravity'][df['Category']=='Silica fume']=2.16
new_df['LOI'][df['Category']=='Silica fume']=6.04


/tmp/ipykernel_2589503/2544071963.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_2589503/2544071963.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [89]:
input_mask = new_df.notnull().astype(int).values
input_mask

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 0, 0, ..., 1, 1, 1],
       [1, 0, 0, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

In [90]:
df_r3 = pd.read_csv('R3.csv')
y = df_r3[['heat release', 'CH consumption', 'Bound water']]

df_r3 = df_r3.drop(columns=['Material name','heat release','CH consumption', 'Bound water'])

for column in df_r3.columns:
    df_r3[column] = pd.to_numeric(df_r3[column], errors='coerce')
    
Updated_compositions = pd.concat([new_df, df_r3], ignore_index=True)
Updated_compositions=Updated_compositions[desired_columns]

In [91]:
Updated_compositions = pd.concat([new_df, df_r3], ignore_index=True)


In [92]:
# Save the 'Category' column into a new variable
categories = df['Category']
matnames = df['Sub_category']

# Drop the 'Category' column
Updated_compositions.head()

,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO2,MnO,K2O,Na2O,...,S gravity,Amorphous content,d50,Ca(OH)2,CaCO3,Water,K2SO4,KOH,curing temp,age
0,64.8,12.77,2.46,NaN,1.84,NaN,0.35,NaN,NaN,NaN,...,NaN,NaN,10.00,3.0,0.0,3.6,0.0,0.151,50,10.0
1,62.3,23.40,4.00,NaN,4.21,NaN,1.3,NaN,NaN,NaN,...,NaN,NaN,10.00,3.0,0.0,3.6,0.0,0.151,50,10.0
2,34.0,12.00,6.00,NaN,13.00,NaN,NaN,NaN,4.00,NaN,...,NaN,NaN,10.00,3.0,0.0,3.6,0.0,0.151,50,10.0
3,0.2,71.00,0.10,28.5,0.40,NaN,NaN,NaN,NaN,0.2,...,NaN,NaN,14.11,3.0,0.0,3.6,0.0,0.151,50,10.0
4,22.1,5.00,3.00,63.8,1.60,2,NaN,NaN,0.64,0.35,...,NaN,NaN,14.11,3.0,0.0,3.6,0.0,0.151,50,10.0


In [94]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Ensure all columns are numeric
for column in Updated_compositions.columns:
    Updated_compositions[column] = pd.to_numeric(Updated_compositions[column], errors='coerce')

def lightgbm_impute(data, target_column, features):
    # Split data into training (known target values) and testing (unknown target values)
    known = data[data[target_column].notna()]
    unknown = data[data[target_column].isna()]

    # If no missing values are present, nothing needs to be done
    if unknown.shape[0] == 0:
        return data

    # Prepare input features and target
    X_train = known[features].values
    y_train = known[target_column].values
    X_test = unknown[features].values

    # Determine if the target column is categorical
    if issubclass(data[target_column].dtype.type, np.integer):
        model = lgb.LGBMClassifier()  # Use classifier for categorical data
    else:
        model = lgb.LGBMRegressor()  # Use regressor for numerical data

    # Train the model
    model.fit(X_train, y_train)

    # Predict the missing values
    predicted_values = model.predict(X_test)

    # Fill in the missing values in the original dataframe without altering the known values
    data.loc[data[target_column].isna(), target_column] = predicted_values

    return data

# Example usage:
# Assuming 'Updated_compositions' is your DataFrame which contains missing values in several columns
columns = Updated_compositions.columns.tolist()

# Impute each column
for column in columns:
    if Updated_compositions[column].isna().any():
        Updated_compositions = lightgbm_impute(Updated_compositions, column, [col for col in columns if col != column])

# Print the imputed DataFrame to verify
print(Updated_compositions)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000781 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3041
[LightGBM] [Info] Number of data points in the train set: 16053, number of used features: 20
[LightGBM] [Info] Start training from score 38.598198
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001216 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3039
[LightGBM] [Info] Number of data points in the train set: 15676, number of used features: 20
[LightGBM] [Info] Start training from score 12.253796
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001034 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3010
[LightGBM] [Info] Number of data points in the train s

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001280 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3528
[LightGBM] [Info] Number of data points in the train set: 16210, number of used features: 20
[LightGBM] [Info] Start training from score 9.480955
       SiO2  Al2O3     Fe2O3        CaO    MgO        SO3      TiO2  \
0      64.8  12.77  2.460000   3.196094   1.84   0.997688  0.350000   
1      62.3  23.40  4.000000   4.035610   4.21   0.834916  1.300000   
2      34.0  12.00  6.000000  23.011334  13.00   5.712305  1.093750   
3       0.2  71.00  0.100000  28.500000   0.40  13.388971 -2.249298   
4      22.1   5.00  3.000000  63.800000   1.60   2.000000  0.223097   
...     ...    ...       ...        ...    ...        ...       ...   
16272  43.3   7.90  1.968469  36.000000   9.80   1.165576  0.600000   
16273  30.2  13.10  1.705107  45.800000   7.38   1.976821  4.512684   
16274  65.5   2.09 

In [95]:
Updated_compositions[Updated_compositions < 0] = 0


In [96]:
number_of_rows_in_df = len(df)

imputed_df = Updated_compositions.iloc[:number_of_rows_in_df]
input_mask = input_mask[:number_of_rows_in_df]


In [97]:
def custom_nan_mse(y_true, y_pred):
    mask = tf.math.logical_not(tf.math.is_nan(y_true))
    mask = tf.cast(mask, dtype=tf.float32)
    
    y_true = tf.where(tf.math.is_nan(y_true), tf.zeros_like(y_true), y_true)
    squared_difference = tf.square(y_true - y_pred) * mask
    
    # Add a small constant (epsilon) to avoid division by zero
    epsilon = 1e-7
    return tf.reduce_sum(squared_difference) / (tf.reduce_sum(mask) + epsilon)

def custom_nan_nrmse(y_true, y_pred):
    mask = tf.math.logical_not(tf.math.is_nan(y_true))
    mask = tf.cast(mask, dtype=tf.float32)
    
    y_true = tf.where(tf.math.is_nan(y_true), tf.zeros_like(y_true), y_true)
    squared_difference = tf.square(y_true - y_pred) * mask
    mse = tf.reduce_sum(squared_difference) / tf.reduce_sum(mask)
    rmse = tf.sqrt(mse)
    
    # Mask y_true values, replacing masked values with infinity and negative infinity
    # to ensure they don't affect the min and max calculations
    y_true_masked_min = tf.where(mask > 0, y_true, tf.fill(tf.shape(y_true), tf.float32.max))
    y_true_masked_max = tf.where(mask > 0, y_true, tf.fill(tf.shape(y_true), tf.float32.min))
    
    # Compute the min and max of the non-NaN y_true values
    y_true_min = tf.reduce_min(y_true_masked_min)
    y_true_max = tf.reduce_max(y_true_masked_max)
    
    y_range = y_true_max - y_true_min
    epsilon = 1e-7
    nrmse = rmse / (y_range + epsilon)
    
    return nrmse




In [98]:
import tensorflow as tf

final_model = tf.keras.models.load_model('90andupnewones.keras', custom_objects={'custom_nan_mse': custom_nan_mse,'custom_nan_nrmse': custom_nan_nrmse})


In [99]:
mean_heat_release = y['heat release'].mean()
std_heat_release = y['heat release'].std()

mean_ch_consumption = y['CH consumption'].mean()
std_ch_consumption = y['CH consumption'].std()

mean_bound_water = y['Bound water'].mean()
std_bound_water = y['Bound water'].std()

In [100]:
import pickle 
with open('df_imputed.pkl', 'rb') as file:
    df_imputed = pickle.load(file)

scaler = StandardScaler()

X_imputed = df_imputed.drop(columns=['heat release', 'CH consumption', 'Bound water'])
X_scaled = scaler.fit_transform(X_imputed)

In [101]:
def unstandardize(values, mean, std):
    """Convert standardized values back to the original scale."""
    return values * std + mean

In [102]:
len(X_scaled[0])

21

In [103]:
X_scaled = scaler.transform(imputed_df)

predictions = final_model.predict([X_scaled,input_mask] )

print(predictions)

451/451 [==============================] - 0s 604us/step
[array([[ 0.3653835 ],
       [-0.1945389 ],
       [-0.4263394 ],
       ...,
       [-0.09584714],
       [-0.2326187 ],
       [-0.82270813]], dtype=float32), array([[ 0.6559918 ],
       [ 0.01792941],
       [-0.9156766 ],
       ...,
       [ 0.2684222 ],
       [ 0.30133402],
       [-0.5613413 ]], dtype=float32), array([[1.0899513],
       [1.5095793],
       [2.438056 ],
       ...,
       [1.2967808],
       [1.3075831],
       [2.2612255]], dtype=float32)]


In [107]:
# Unstandardize predictions
predictions_unstd = [
    unstandardize(predictions[0].flatten(), mean_heat_release, std_heat_release),
    unstandardize(predictions[1].flatten(), mean_ch_consumption, std_ch_consumption),
    unstandardize(predictions[2].flatten(), mean_bound_water, std_bound_water)
]

In [108]:
ch_consumption_arr = np.array(predictions_unstd[1])
bound_water_arr = np.array(predictions_unstd[2])
heat_release_arr = np.array(predictions_unstd[0])


# Find indices where the value is greater than 380
indices_heat = np.where(heat_release_arr > 380)[0]
# Find indices where bound_water_arr > 10
indices_water = np.where(bound_water_arr > 10)[0]

# Find common indices satisfying both conditions
# First, intersect the first two arrays
common_indices = np.intersect1d(indices_heat, indices_water)


df['Heat release']=heat_release_arr
df['CH consumption']=ch_consumption_arr
df['Bound water']=bound_water_arr

# Filter the dataframe using these indices
filtered_df = df.iloc[common_indices]

# Display the filtered dataframe
print(filtered_df)

          Material Name   SiO2  Al2O3  Fe2O3    CaO    MgO   SO3 TiO2  MnO  \
3                   HAC   0.20  71.00   0.10  28.50   0.40   NaN  NaN  NaN   
38                   MK  52.00  40.00   2.50   1.00   0.80   NaN  NaN  NaN   
50            Residuals  47.60  37.00   0.80   9.40   2.00   NaN  4.9  NaN   
55                 Slag  32.02  16.15   0.84  38.78   8.40  0.23  NaN  NaN   
80       Granulated BFS  33.78  13.11   0.51  42.47   7.46  0.15  NaN  NaN   
...                 ...    ...    ...    ...    ...    ...   ...  ...  ...   
14393            MK [%]  54.77  41.01   1.51   0.19   0.52   NaN  NaN  NaN   
14409           Al-ULCC   4.90  94.20    NaN   0.30    NaN   NaN  NaN  NaN   
14417               T75   0.65  34.37   0.50   0.97  63.35   NaN  NaN  NaN   
14420               T25   0.82  11.62   0.64   1.30  85.57   NaN  NaN  NaN   
14421  Alumina A1000-SG   0.03  99.90   0.02   0.02    NaN   NaN  NaN  NaN   

        K2O  ...                                         Descri